In [1]:
import datetime
import mysql.connector
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from tabulate import tabulate
import openpyxl
import pandas as pd
from dateutil.parser import parse
import re

#Libreria de fechas
from datetime import datetime

host = 'localhost'
user = 'root'
password = 'Estadistica123'
database = 'prueba1'

#Var que contiene el dataframe generado 
df_final = pd.DataFrame()

class loadHTML_TablaAutoNacion:
    def loadInDataBase(self, host, user, password, database):
        # Se toma el tiempo de comienzo
        start_time = time.time()

        # Establecer la conexión a la base de datos
        conn = mysql.connector.connect(
            host=host, user=user, password=password, database=database
        )
        # Crear el cursor para ejecutar consultas
        cursor = conn.cursor()
        try:
            ruta_archivo_excel = 'C:\\Users\\Usuario\\Desktop\\scrapingTrabajo\\script_DNRPA\\prueba.xlsx' 
            #ruta_archivo_excel = 'D:\\Users\\Pc-Pix211\\Desktop\\scrapingTrabajo\\script_DNRPA\\prueba.xlsx'
            driver = webdriver.Chrome()
            driver.get('https://www.dnrpa.gov.ar/portal_dnrpa/estadisticas/rrss_tramites/tram_prov.php?origen=portal_dnrpa&tipo_consulta=inscripciones')

            # Obtener la ventana actual
            ventana_actual = driver.current_window_handle
            
            elemento = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/table/tbody/tr[2]/td/select')
            # Obtener todas las opciones del elemento select
            opciones = elemento.find_elements(By.TAG_NAME, 'option')

            # Buscar la opción deseada por su valor y hacer clic en ella
            valor_deseado = '2014'  # Valor de la opción que deseas seleccionar

            for opcion in opciones: #Proceso de 'Click' sobre el boton 'aceptar'
                if opcion.get_attribute('value') == valor_deseado:
                    opcion.click()
                    break
            
            boton = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/table/tbody/tr[4]/td/input[1]')
            boton.click()
            
            time.sleep(5)
            
            boton_aceptar = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/center/input')
            boton_aceptar.click()
            
            # Esperar un momento para que se abra la nueva pestaña
            driver.implicitly_wait(5)
            # Cambiar al contexto de la nueva pestaña
            for ventana in driver.window_handles:
                if ventana != ventana_actual:
                    driver.switch_to.window(ventana)
            
            time.sleep(5)

            # Encontrar el elemento <div> con la clase 'grid'
            elemento_div = driver.find_element(By.CLASS_NAME, 'grid')

            # Encontrar la tabla dentro del elemento <div>
            elemento_tabla = elemento_div.find_element(By.TAG_NAME, 'table')

            # Obtener todas las filas de la tabla
            filas = elemento_tabla.find_elements(By.TAG_NAME, 'tr')

            # Lista para almacenar los datos de la tabla
            tabla_datos = []
            
            for fila in filas:
                # Obtener las celdas de cada fila, excluyendo la última columna y la última celda de encabezado
                celdas = fila.find_elements(By.TAG_NAME, 'th') + fila.find_elements(By.TAG_NAME, 'td')[:-1]

                # Lista para almacenar los valores de cada fila
                fila_datos = []

                for celda in celdas:
                    valor = celda.text
                    if isinstance(valor, str):
                        # Verificar si el valor comienza con un número
                        if valor.strip() and valor[0].isdigit():
                            try:
                                # Reemplazar el punto decimal por una coma (si es necesario)
                                valor = valor.replace('.', '')
                                # Intentar convertir el valor a float
                                valor = int(valor)
                            except ValueError:
                                pass  # Mantener el valor original si no se puede convertir a float
                    fila_datos.append(valor)

                # Verificar si la última celda es "Total" y eliminarla
                if fila_datos and fila_datos[-1] == "Total":
                    fila_datos.pop()

                # Agregar la lista de datos de la fila a la tabla de datos
                tabla_datos.append(fila_datos) 
            
            #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓FUNCIONA BIEN MODIFICAR LO DE ABAJO ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
            datos_sin_segunda_fila = tabla_datos[0:1] + tabla_datos[2:]
            # Transponer los datos utilizando pandas
            df = pd.DataFrame(datos_sin_segunda_fila)
            df_transpuesta = df.transpose()
            

            # ==== SECCION DE CONVERSION A FECHAS DE LA COLUMNA ' PROVINCIA / MES ' ==== #

            #Conversion de MESES a formato Y-M-D , tipo de dato: datetime
            print(df_transpuesta[0][1:])
            meses = df_transpuesta[0][1:]

            

            #Donde almacenamos las nuevas fechas
            nuevas_fechas = list()

            for i in range(1, len(meses)+ 1):

                if i < 10:
                    fecha_str =  '01-0'+str(i)+"-"+ str(valor_deseado)
                else:
                    fecha_str = '01-'+str(i)+"-"+ str(valor_deseado)

            
                fecha_str = datetime.strptime(fecha_str,'%d-%m-%Y').date()

                nuevas_fechas.append(fecha_str)

            
            #Reasignacion de fechas

            df_transpuesta[0][1:] = nuevas_fechas

            #Lllamamos a la variable global para visualizar los datos del DF final
            global df_final
            df_final = df_transpuesta

            # Convertir los valores de la transposición a una lista
            valores_transpuestos = df_transpuesta.values.tolist()

            # Cargar el archivo Excel existente
            libro_excel = openpyxl.load_workbook(ruta_archivo_excel)

           # Seleccionar la hoja activa del libro
            hoja_activa = libro_excel.active

            # Obtener la última fila existente en el archivo
            ultima_fila = hoja_activa.max_row + 1

            # Recorrer los datos transpuestos y escribirlos en el archivo de Excel
            for fila_datos in valores_transpuestos:
                hoja_activa.append(fila_datos)
                
            df_transpuesta.drop
            
            # Guardar el archivo Excel actualizado sobreescribiendo los datos existentes
            libro_excel.save(ruta_archivo_excel)

            # Se toma el tiempo de finalización y se calcula
            end_time = time.time()
            duration = end_time - start_time
            print("-----------------------------------------------")
            print("Se guardaron los datos de SIPA NACIONAL CON ESTACIONALIDAD")
            print("Tiempo de ejecución:", duration)

            # Cerrar la conexión a la base de datos
            conn.close()

        except Exception as e:
            # Manejar cualquier excepción ocurrida durante la carga de datos
            print(f"Data Cuyo: Ocurrió un error durante la carga de datos: {str(e)}")
            conn.close()  # Cerrar la conexión en caso de error
            
loadHTML_TablaAutoNacion().loadInDataBase(host, user, password, database)


1     Ene
2     Feb
3     Mar
4     Abr
5     May
6     Jun
7     Jul
8     Ago
9     Sep
10    Oct
11    Nov
12    Dic
Name: 0, dtype: object
-----------------------------------------------
Se guardaron los datos de SIPA NACIONAL CON ESTACIONALIDAD
Tiempo de ejecución: 167.86121797561646


In [2]:
print(df_final)

                 0             1                    2          3        4   \
0   Provincia / Mes  BUENOS AIRES  C.AUTONOMA DE BS.AS  CATAMARCA  CORDOBA   
1        2014-01-01         35306                15737       1036    10692   
2        2014-02-01         19362                 9521        426     5474   
3        2014-03-01         16892                 8638        429     5103   
4        2014-04-01         17525                 8369        414     5335   
5        2014-05-01         18310                 9189        498     5526   
6        2014-06-01         16704                 8687        451     5470   
7        2014-07-01         19624                 9577        535     6517   
8        2014-08-01         18820                 8805        477     5795   
9        2014-09-01         18495                 9712        517     5840   
10       2014-10-01         17118                 8826        439     5271   
11       2014-11-01         12187                 6905        28

In [4]:
#Visualizacion de datos
print(df_final[0])

"""
Existencia de imcompatiblidad

Fecha Excel: formato Y/M/D --> No acepta otro formato
Fecha Python: formayo Y-M-D

"""

#Tipo de dato tratado
print("====================")
print("Tipo de dato tratado por la Serie" + str(type(df_final[0][1])))

0     Provincia / Mes
1          2014-01-01
2          2014-02-01
3          2014-03-01
4          2014-04-01
5          2014-05-01
6          2014-06-01
7          2014-07-01
8          2014-08-01
9          2014-09-01
10         2014-10-01
11         2014-11-01
12         2014-12-01
Name: 0, dtype: object
Tipo de dato tratado por la Serie<class 'datetime.date'>
